In [50]:
geom_root = '/sharefs/baaihealth/yuancheng/rdkit_folder/qm9'
import glob
import os.path as osp
all_files = sorted(glob.glob(osp.join(geom_root, '*.pickle')))
import pickle
import glob
import numpy as np
import random
import torch
# with open(all_files[-1], 'rb') as f:
#     dic = pickle.load(f)
# confs = dic['conformers']



In [51]:
def _get_relative_pos(qm9_graph):
    x = qm9_graph.ndata['pos']
    src, dst = qm9_graph.edges()
    rel_pos = x[dst] - x[src]
    return rel_pos

import dgl
batched_graph = dgl.batch([graph])

batched_graph.edata['rel_pos'] = _get_relative_pos(batched_graph)
edge_feats = {'0': batched_graph.edata['edge_attr'][..., None]}
node_feats = {'0': batched_graph.ndata['attr'][:, :, None]}

# return batched_graph, node_feats, edge_feats, targets


Using backend: pytorch


NameError: name 'graph' is not defined

In [ ]:
from se3 import SE3Transformer, SE3TransformerPooled
from se3 import Fiber

num_degrees=4
num_channels=32

num_layers=7
num_heads=8
channels_div=2
norm=True
use_layer_norm=True
low_memory=False


toymodel = SE3Transformer(
    fiber_in=Fiber({0: 9}),
    fiber_hidden=Fiber.create(num_degrees, num_channels),
    fiber_out=Fiber({0: num_degrees * num_channels}),
    fiber_edge=Fiber({0: 3}),
    tensor_cores=True,
    num_layers=num_layers,
    num_heads=num_heads,
    channels_div=channels_div,
    norm=norm,
    use_layer_norm=use_layer_norm,
    low_memory=low_memory
)


toymodel_pooled = SE3Transformer(
    fiber_in=Fiber({0: 9}),
    fiber_hidden=Fiber.create(num_degrees, num_channels),
    fiber_out=Fiber({0: num_degrees * num_channels}),
    fiber_edge=Fiber({0: 3}),
    tensor_cores=True,
    num_layers=num_layers,
    num_heads=num_heads,
    channels_div=channels_div,
    norm=norm,
    use_layer_norm=use_layer_norm,
    low_memory=low_memory,
    pooling='avg',
    return_type=1
)

In [ ]:
res = toymodel.forward(graph=batched_graph, node_feats=node_feats, edge_feats=edge_feats, basis=None)


In [12]:
import torch

angle_mask_ref = torch.LongTensor([[0, 0, 0, 0, 0, 0],
                                   [0, 0, 0, 0, 0, 0],
                                   [1, 0, 0, 0, 0, 0],
                                   [1, 1, 1, 0, 0, 0],
                                   [1, 1, 1, 1, 1, 1]])


angle_combos = torch.LongTensor([[0, 1],
                                 [0, 2],
                                 [1, 2],
                                 [0, 3],
                                 [1, 3],
                                 [2, 3]])


mask = torch.tensor([0,1,1,1,1,1,1,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,0,0,1,1,1,0,1,0]).view([-1,4])
coords = torch.rand([16,6,4,10,3])
all_possible_combos = coords[:, :, angle_combos]
v_a, v_b = all_possible_combos.split(1, dim=3) 
angle_mask = angle_mask_ref[mask.sum(dim=1).long()]



In [17]:
pos = torch.randn(100, 3)
local_coords = torch.zeros(150, 4, 3)
for i, n in enumerate(local_coords):
    if i >= 100:
        i = 99
    local_coords[i,:] = pos[i]


In [49]:
coords = torch.randn([32,6,4,10,3])
all_possible_combos = coords[:,:, angle_combos]

# print(coords.shape)
# print(all_possible_combos.shape)
v_a, v_b = all_possible_combos.split(1, dim=3)  # does one of these need to be negative?
print(v_a.squeeze(3).shape)
angle_mask = angle_mask_ref[mask.sum(dim=1).long()]
# angles = batch_angle_between_vectors(v_a.squeeze(3), v_b.squeeze(3)) * angle_mask.unsqueeze(-1).unsqueeze(-1)
print(batch_angle_between_vectors(v_a.squeeze(3), v_b.squeeze(3)).shape)
print(angle_mask.unsqueeze(-1).unsqueeze(-1).shape)
print(angle_mask)

torch.Size([32, 6, 6, 10, 3])


IndexError: index 7 is out of bounds for dimension 0 with size 5

In [41]:
qm_coor = torch.rand(32,4,3)
qm_coor_all = qm_coor[:,  angle_combos]
# print(qm_coor_all.shape)
v_a, v_b = qm_coor_all.split(1, dim=2)
angle_mask = angle_mask_ref[mask.sum(dim=1).long()]
    # angles = batch_angle_between_vectors(v_a.squeeze(2), v_b.squeeze(2)) * angle_mask.unsqueeze(-1)
print(batch_angle_between_vectors(v_a.squeeze(2), v_b.squeeze(2)).shape)
print(angle_mask.shape)

torch.Size([32, 6])
torch.Size([10, 6])


In [37]:
def batch_angle_between_vectors(a, b):
    """
    Compute angle between two batches of input vectors
    """
    inner_product = (a * b).sum(dim=-1)

    # norms
    a_norm = torch.linalg.norm(a, dim=-1)
    b_norm = torch.linalg.norm(b, dim=-1)

    # protect denominator during division
    den = a_norm * b_norm + 1e-10
    cos = inner_product / den

    return cos


In [9]:
mask

tensor([[0, 1, 1, 2],
        [2, 2, 3, 3],
        [3, 3, 4, 4],
        [4, 4, 4, 5],
        [5, 5, 5, 5],
        [5, 6, 6, 6],
        [6, 6, 6, 6],
        [7, 7, 7, 7],
        [7, 7, 7, 7]])